In [1]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets

In [2]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [3]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

UNCAUGHT SHEET DETECTED:  <Worksheet 'Import' index:6>
UNCAUGHT SHEET DETECTED:  <Worksheet 'Lifetime Deliveries' index:7>
UNCAUGHT SHEET DETECTED:  <Worksheet 'Days Old' index:8>
UNCAUGHT SHEET DETECTED:  <Worksheet 'Weeks Old' index:9>


In [5]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [6]:
# FUNCTIONS #

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            

#Currently only valid for 2023
def date_to_timestamp(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
        
        
#Currently only valid for 2023
#Update to handle orders that happen past midnight
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [7]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    testingDF = testingDF.sort_values(by=['Restaurant Name', "RAW DATA"])
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])        
    #newNamesDF.to_csv('RestGroups.csv', index=False)
    wks_gps.set_dataframe(newNamesDF,(1,1))

In [8]:
# COMBINE DELIVERY DATA #
sheets_frames = [sheets_OldDelDF, sheets_DelDF]
sheets_result = pd.concat(sheets_frames)
sheets_result.reset_index(drop=True, inplace=True)

In [9]:
# FUNCTIONS TO CONVERT DOLLAR AND HOUR VALUES TO FLOATS #


#All
dollars_to_floats(sheets_result, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])

/tmp/ipykernel_19046/1965361392.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [10]:
# BEGIN ADJ PAY SECTION #

sheets_DelDF.tail()

,ID,Date,Restaurant Name,Total,Base,Tip,Peak Bonus,Stacked,Start Time,Distance,End Time,Duration,Version
447,1022,7/29,El Pollo Loco,6.25,3.25,3.0,0.0,,7:58:00 PM,1.7,8:11:00 PM,13,3
448,1023,7/29,Disfruta,5.75,3.75,2.0,0.0,,7:37:00 PM,2.4,7:51:00 PM,14,3
449,1024,7/29,Best pizza and brew,13.00,6.50,6.5,0.0,1025,9:30:00 PM,11.9,10:10:00 PM,40,3
450,1025,7/29,Panda Express,9.50,6.50,3.0,0.0,1024,9:30:00 PM,11.9,10:10:00 PM,40,3
451,1026,7/29,McDonald's,9.00,5.50,3.5,0.0,,6:41:00 PM,2.1,7:01:00 PM,20,3


In [11]:
#Convert to float, all non converted yield NaN
sheets_DelDF["Duration"] = pd.to_numeric(sheets_DelDF["Duration"], errors='coerce')
#Drop NaN
sheets_DelDF = sheets_DelDF[sheets_DelDF["Duration"].notna()]


#sheets_DelDF["Duration"] = sheets_DelDF["Duration"].astype(float)
sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18 
                            + sheets_DelDF["Distance"]*(0.34) ) - sheets_DelDF["Base"]
    
sheets_DelDF.tail()

/tmp/ipykernel_19046/2053155319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18


,ID,Date,Restaurant Name,Total,Base,Tip,Peak Bonus,Stacked,Start Time,Distance,End Time,Duration,Version,Est Adj
447,1022,7/29,El Pollo Loco,6.25,3.25,3.0,0.0,,7:58:00 PM,1.7,8:11:00 PM,13.0,3,1.228
448,1023,7/29,Disfruta,5.75,3.75,2.0,0.0,,7:37:00 PM,2.4,7:51:00 PM,14.0,3,1.266
449,1024,7/29,Best pizza and brew,13.00,6.50,6.5,0.0,1025,9:30:00 PM,11.9,10:10:00 PM,40.0,3,9.546
450,1025,7/29,Panda Express,9.50,6.50,3.0,0.0,1024,9:30:00 PM,11.9,10:10:00 PM,40.0,3,9.546
451,1026,7/29,McDonald's,9.00,5.50,3.5,0.0,,6:41:00 PM,2.1,7:01:00 PM,20.0,3,1.214


In [12]:
sheets_DaysDF.tail()
#sheets_DaysDF.dtypes

,Date,Total,Base,Tip,Start (24),End (24),Active,Dash,Offers,Deliveries,Mileage Start,Mileage End,Total Mileage,Active Mileage,Version,Pauses
53,7/23,201.00,102.00,99.00,12:39,20:23,5.18,7.73,21,21,70658,70769,111,103.2,3,"(5:18pm, 5:41pm)"
54,7/26,103.00,55.00,48.00,17:25,21:37,3.03,4.18,11,11,70831,70887,56,50.0,3,"(7:30pm, 7:50pm)"
55,7/27,111.55,67.25,44.30,17:33,22:08,3.58,4.57,12,12,70900,70957,57,47.4,3,0
56,7/28,162.24,89.25,72.99,15:06,21:41,4.62,6.57,14,15,70962,71071,109,90.1,3,"(6:27pm, 6:57pm)"
57,7/29,111.25,66.75,44.50,18:00,20:10,3.55,4.15,10,12,71086,71149,63,84.3,3,0


In [13]:
#Interesting, I don't think peak pay is factored into pay adjustments, but worth check
sheets_DaysDFv3 = sheets_DaysDF.copy()
sheets_DaysDFv3 = sheets_DaysDFv3[sheets_DaysDFv3["Version"] == 3]
sheets_DaysDFv3["Est Adj"] = (sheets_DaysDFv3["Active"]*(18) 
                            + sheets_DaysDFv3["Active Mileage"]*(0.34) ) - sheets_DaysDFv3["Base"]

In [14]:
sheets_DaysDFv3.tail()

,Date,Total,Base,Tip,Start (24),End (24),Active,Dash,Offers,Deliveries,Mileage Start,Mileage End,Total Mileage,Active Mileage,Version,Pauses,Est Adj
53,7/23,201.00,102.00,99.00,12:39,20:23,5.18,7.73,21,21,70658,70769,111,103.2,3,"(5:18pm, 5:41pm)",26.328
54,7/26,103.00,55.00,48.00,17:25,21:37,3.03,4.18,11,11,70831,70887,56,50.0,3,"(7:30pm, 7:50pm)",16.54
55,7/27,111.55,67.25,44.30,17:33,22:08,3.58,4.57,12,12,70900,70957,57,47.4,3,0,13.306
56,7/28,162.24,89.25,72.99,15:06,21:41,4.62,6.57,14,15,70962,71071,109,90.1,3,"(6:27pm, 6:57pm)",24.544
57,7/29,111.25,66.75,44.50,18:00,20:10,3.55,4.15,10,12,71086,71149,63,84.3,3,0,25.812


In [15]:
sheets_WeeksDFv3 = sheets_WeeksDF.copy()
sheets_WeeksDFv3 = sheets_WeeksDFv3[sheets_WeeksDF["Version"] == 3]
sheets_WeeksDFv3["Est Adj"] = (sheets_WeeksDFv3["Active"]*(18) 
                            + sheets_WeeksDFv3["Active Mileage"]*(0.34) ) - sheets_WeeksDFv3["Base"]

In [16]:
sheets_WeeksDFv3.tail()

# END ADJ PAY SECTION #

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,Est Adj
11,6/12 - 6/18,116.00,101.05,34.67,6.50,217.05,258.22,6.43,9.57,21,147,113.6,3,38.364
12,6/19 - 6/25,637.25,475.09,240.02,6.25,1112.34,1358.61,38.60,51.23,124,633,619.4,3,268.146
13,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,60.044
14,7/17 - 7/23,454.75,425.11,151.31,3.38,879.86,1034.55,26.22,37.30,91,471,438.6,3,166.334
15,7/24 - 7/30,278.25,209.79,0.00,0.00,488.04,488.04,11.27,15.35,50,285,271.8,3,17.022


In [17]:
#Percentage of customers who tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 3)
print(percTipped*100, "% of customers tip")

90.60000000000001 % of customers tip


In [18]:
tempDF = sheets_result.copy()
tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

,Base,Tip
Restaurant Name,,
Albertsons,4.583333,12.39
Hunter Steakhouse,10.666667,10.33
L&L Hawaiian Barbecue,6.750000,9.50
Chin's Szechwan,5.875000,9.25
Mangia e Bevi,6.250000,9.00
...,...,...
Lollicup Fresh,4.250000,1.88
Carl's Jr.,4.416667,1.33
Sizzler,12.750000,1.25


In [19]:
tempDF = sheets_WeeksDF.copy()
#double bracket on iloc returns a df instead of series
tempDF = tempDF.iloc[[-1]]
#dollars_to_floats(tempDF, ["Total"])
#hours_to_floats(tempDF, ["Dash"])
hourlyRate = tempDF["Total"]/tempDF["Dash"]
float(tempDF["Dash"])
#Hourly rate is touple? just print [1] then
print("$", hourlyRate)

$ 16    25.547876
dtype: float64


In [20]:
#So float(var) converts, but series.astype(float) is for series in df

In [21]:
#dollars_to_floats(sheets_WeeksDF, ["Pre Total"])
#hours_to_floats(sheets_WeeksDF, ["Active"])

sheets_WeeksDF["pretotalactiveratio"] = sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"]
sheets_WeeksDF.sort_values(by=["pretotalactiveratio"])

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,pretotalactiveratio
2,4/10 - 4/16,130.75,96.13,61.07,0.00,226.88,287.95,8.63,11.17,28,,,1,26.289687
13,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,28.014049
4,4/24 - 4/30,430.00,299.73,138.76,2.62,729.73,871.11,25.50,41.17,84,,,1,28.616863
12,6/19 - 6/25,637.25,475.09,240.02,6.25,1112.34,1358.61,38.60,51.23,124,633,619.4,3,28.817098
0,3/27 - 4/2,108.50,88.00,40.96,0.00,196.50,237.46,6.70,10.92,22,,,1,29.328358
3,4/17 - 4/23,382.50,248.41,105.05,0.00,630.91,735.96,21.33,32.30,68,,,1,29.578528
7,5/15 - 5/21,303.75,239.25,99.27,0.00,543.00,642.27,17.53,25.40,64,,,2,30.975471
16,TOTALS:,5342.50,4215.67,1505.80,32.75,9558.17,11096.72,300.53,434.35,1024,2446,2229.68,-,31.804379
10,6/5 - 6/11,197.25,185.09,65.38,0.00,382.34,447.72,11.77,15.75,41,183,161.6,3,32.484282
6,5/8 - 5/14,758.25,567.84,155.48,6.25,1326.09,1487.82,40.58,58.13,138,,,2,32.678413


In [22]:
#Dollars Earned vs Miles Driven doodle
#$ to float
#drop non numeric Miles values, then convert column to float
#agg sum Total and Miles, by restaurant name (then round)
#create new column as Total/Miles
#sort low to high
tempDF = sheets_DelDF.copy()
#dollars_to_floats(tempDF, ["Total"])
#display(tempDF)
tempDF = tempDF[tempDF.Distance!="-"]
tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
tempDF.sort_values(by="DollarMilesRatio")

,Total,Distance,DollarMilesRatio
Restaurant Name,,,
Yukiya Sushi,8.25,10.8,0.763889
Pedro's Tacos,7.00,7.9,0.886076
El Bronco,9.25,9.7,0.953608
Papa Johns,5.00,4.9,1.020408
Urban Plates,21.50,20.4,1.053922
...,...,...,...
The Privateer Coal Fire Pizza,8.00,1.8,4.444444
Ohana Hawaiian BBQ,8.25,1.8,4.583333
Round Table Pizza,9.75,1.4,6.964286


In [23]:
sheets_DaysDF.max() #would be useful with more attributes converted to float 

/tmp/ipykernel_19046/842620842.py:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  sheets_DaysDF.max() #would be useful with more attributes converted to float


Date             7/8
Total         333.57
Base           191.5
Tip           149.53
Start (24)     18:58
End (24)        2:32
Active          9.67
Dash            13.6
Offers            32
Deliveries        33
Version            3
dtype: object

In [24]:
#Avg tip vs Avg tip excluding zero tip orders

TipAvg = sheets_result["Tip"].mean()
print(TipAvg)
TipAvg2 = sheets_result[sheets_result["Tip"] !=0]["Tip"].mean()
print(TipAvg2)

4.112848780487805
4.537857911733046


In [25]:
#CONVERT STRING OF DATE TO TIMESTAMP, THEN TIMESTAMP TO DATETIME
date_to_timestamp(sheets_DaysDF, "Date")
sheets_DaysDF["Date"] = pd.to_datetime(sheets_DaysDF["Date"])

In [26]:
#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()

In [27]:
#hours_to_floats(sheets_DaysDF, ["Dash"])
#dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])

In [28]:
sheets_DaysDF = sheets_DaysDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", "Total": "sum", "Tip": "sum", "Deliveries": "sum"})
sheets_DaysDF = sheets_DaysDF.round({'Dash': 2, 'Total': 2, "Tip": 2})
sheets_DaysDF["$/hr"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash"]
sheets_DaysDF["AvgTip"] = sheets_DaysDF["Tip"]/sheets_DaysDF["Deliveries"]
display(sheets_DaysDF)

,Dash,Total,Tip,Deliveries,$/hr,AvgTip
Day of Week,,,,,,
Friday,127.99,2869.09,1318.34,301,22.416517,4.379867
Monday,17.85,377.47,158.97,41,21.146779,3.877317
Saturday,106.08,2422.70,1067.45,252,22.838424,4.235913
Sunday,45.36,1083.78,507.03,111,23.892857,4.567838
Thursday,72.83,1548.51,656.01,176,21.261980,3.727330
Wednesday,67.81,1256.62,507.87,143,18.531485,3.551538
